<a href="https://colab.research.google.com/github/rizqinugroho/learning-hadoop/blob/main/learning-spark/DataAnalytic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Spark ditulis dalam bahasa Scala dan membutuhkan Java Virtual Machine (JVM) untuk bisa berjalan. Maka dari itu, kita membutuhkan java untuk dia bisa berjalan diatasnya

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
!tar xf spark-3.2.1-bin-hadoop2.7.tgz

set the ‘environment’ path.

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7/bin"

Lalu, kita. butuh menginstall **Findspark** library yang akan mencari spark dalam sistem dan akan menginstallnya sebagai regular library.  

In [6]:
!pip install -q findspark
import findspark
findspark.init("/content/spark-3.2.1-bin-hadoop2.7/", edit_rc=True)

Sekarang, kita bisa import SparkSession dari pyspark.sql and membuat  SparkSession, yang merupakan pintu gerbang to Spark.

In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Download Sample dataset 

In [ ]:
!wget --continue https://raw.githubusercontent.com/rizqinugroho/learning-hadoop/main/learning-spark/sample-books.json -O /tmp/sample-book.json

--2022-02-02 13:05:02--  https://raw.githubusercontent.com/rizqinugroho/learning-hadoop/main/learning-spark/sample-books.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1578 (1.5K) [text/plain]
Saving to: ‘/tmp/sample-book.json’

/tmp/sample-book.js 100%[===================>]   1.54K  --.-KB/s    in 0s      

2022-02-02 13:05:02 (20.6 MB/s) - ‘/tmp/sample-book.json’ saved [1578/1578]



Buat sebuah dataframe dari sample file

In [ ]:
df = spark.read.option("multiline","true").json("/tmp/sample-book.json")
df.show()

NameError: ignored

Print schema

In [ ]:
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [ ]:
df_lenght = df.count()
print(df_lenght)


13


In [ ]:
df.count()

13

In [ ]:
df.select("title", "price", "year_written").show(5)

+----------------+-----+------------+
|           title|price|year_written|
+----------------+-----+------------+
|Northanger Abbey| 18.2|        1814|
|   War and Peace| 12.7|        1865|
|   Anna Karenina| 13.5|        1875|
|   Mrs. Dalloway| 25.0|        1925|
|       The Hours|12.35|        1999|
+----------------+-----+------------+
only showing top 5 rows



Salah satu contoh narrow transformation adalah filter

In [ ]:
df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.select("title", "price", "year_written").show(50, truncate=False)

+-----------------------------+-----+------------+
|title                        |price|year_written|
+-----------------------------+-----+------------+
|The Hours                    |12.35|1999        |
|Harry Potter                 |19.95|2000        |
|One Hundred Years of Solitude|14.0 |1967        |
+-----------------------------+-----+------------+



Spark transformation adalah lazzy maka saat kita mendefine satu syntax dia tidak akan langsung di eksekusi

In [ ]:
df_filtered_tittle = df_filtered.filter("year_written=2000")
df.show()
df_filtered.show()
df_filtered_tittle.show()


+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

Eksekusi hanya akan berjalan setelah ada action, berikut sample action .show()

In [ ]:
modulus.select("title", "price", "year_written").show()

+------------+-----+------------+
|       title|price|year_written|
+------------+-----+------------+
|Harry Potter|19.95|        2000|
+------------+-----+------------+



Sample wide transformation adalah groupBy

In [ ]:
df.groupby('edition').count().show()

+-----------------+-----+
|          edition|count|
+-----------------+-----+
| Signet  Classics|    1|
|Harper  Perennial|    1|
|     Random House|    2|
|   Harcourt Brace|    3|
|          Penguin|    6|
+-----------------+-----+



Untuk mengetahui lebih dalam, kenapa wide gunakan explain

In [ ]:
grp_by = df.groupby('edition').count()
grp_by.explain()

df_filtered = df.filter("year_written > 1950 AND price > 10 AND title IS NOT NULL")
df_filtered.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[edition#47], functions=[count(1)])
   +- Exchange hashpartitioning(edition#47, 200), ENSURE_REQUIREMENTS, [id=#224]
      +- HashAggregate(keys=[edition#47], functions=[partial_count(1)])
         +- FileScan json [edition#47] Batched: false, DataFilters: [], Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/tmp/sample-book.json], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<edition:string>


== Physical Plan ==
*(1) Filter ((((isnotnull(year_written#50L) AND isnotnull(price#48)) AND (year_written#50L > 1950)) AND (price#48 > 10.0)) AND isnotnull(title#49))
+- FileScan json [author#46,edition#47,price#48,title#49,year_written#50L] Batched: false, DataFilters: [isnotnull(year_written#50L), isnotnull(price#48), (year_written#50L > 1950), (price#48 > 10.0), ..., Format: JSON, Location: InMemoryFileIndex(1 paths)[file:/tmp/sample-book.json], PartitionFilters: [], PushedFilters: [IsNotNull(y